In [ ]:
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

g = nx.Graph()
g.add_edges_from(
    [(1, 2), (1, 9), (3, 13), (4, 6),
    (5, 6), (5, 7), (5, 8), (5, 9),
    (5, 10), (5, 11), (5, 12), (10, 12),
    (10, 13), (11, 14), (12, 14),
    (12, 15), (13, 14), (13, 15),
    (13, 17), (14, 15), (15, 16)]
)

In [ ]:
gg = nx.core.k_shell(g)
nx.draw(gg, with_labels=True)

In [ ]:
import network_diffusion as nd
net = nd.MultilayerNetwork.load_mlx(file_path="data/aucs.mpx")
fb = net.layers["facebook"]

In [ ]:
gg = fb.copy()
fig, axes = plt.subplots(nrows=3, ncols=3)
row = -1
col = -1

for k in range(9):

    row += 1
    if row % 3 == 0:
        row = 0
        col += 1

    axis = axes[col][row]
    axis.set_title(f"K={k}")

    ggg = nx.k_shell(gg, k=k)
    
    nx.draw(ggg, with_labels=True, ax=axis)
    print(k, [[node, nx.degree(gg, node)] for node in ggg.nodes()])

    if len(list(gg.nodes())) == 0:
        break

fig.set_size_inches(18.5, 10.5)
fig.tight_layout()

In [ ]:
nx.draw(sh:=nx.k_shell(fb), with_labels=True)
print(k, [[node, nx.degree(fb, node)] for node in sh.nodes()])

In [ ]:
nx.draw(fb, with_labels=True)

In [ ]:
def create_k_shell_list(g: nx.Graph):
    ksh_deepest_nodes = set(nx.k_shell(g).nodes())
    shell_ranking = {}
    k = 0

    while True:
        ksh_nodes = set(nx.k_shell(g, k=k).nodes())
        shell_ranking[k] = ksh_nodes
        if ksh_nodes == ksh_deepest_nodes:
            break
        k += 1
    
    return shell_ranking

In [ ]:
rl = create_k_shell_list(fb)

In [ ]:
print(sorted(rl)[::-1])

In [ ]:
rl

In [ ]:
ll = [list(rl[k]) for k in sorted(rl)[::-1]]
import itertools

In [ ]:
lll = [n for cohort in sorted(rl)[::-1] for n in rl[cohort]]

In [ ]:
lll

### Degree centrality

In [1]:
import network_diffusion as nd
import networkx as nx

%load_ext autoreload
%autoreload 2

In [ ]:
layer_1 = nx.Graph(
    (
        [1, 4], [2, 4], [2, 3], [3, 4], [3, 5],
        [3, 6], [4, 5], [7, 9], [8, 9], [8, 10]
    )
)

layer_2 = nx.Graph(
    (
        [1, 2], [2, 7], [2, 11], [4, 5], [4, 6],
        [5, 11], [6, 10], [7, 9], [8, 9], [8, 10],
        [10, 11]
    )
)

layer_3 = nx.Graph(
    (
        [1, 4], [2, 6], [2, 9], [3, 4], [3, 5],
        [4, 5], [5, 6], [5, 11], [6, 9], [7, 9], [10, 11]
    )
)

net = nd.MultilayerNetwork.load_layers_nx([layer_1, layer_2, layer_3], ["l1", "l2", "l3"])

In [ ]:
print(net.get_actors_num())
for actor in net.get_actors():
    print(actor)

In [ ]:
selector = nd.seeding.PageRankSeedSelector()
ranking = selector(net, actorwise=True)
print([r.actor_id for r in ranking])

In [ ]:
nx.pagerank(net.layers["l1"])

In [ ]:
my_dict = { 'num6': 6, "34": 2, 'num2': 2, 'num4': 4, 'num1': 1, 'num5': 5}
sortedDict = sorted(my_dict.values())

In [ ]:
sortedDict

In [ ]:
import numpy as np

In [ ]:
seeding_budgets = np.arange(0, 101, 1)

In [ ]:
s = [(a, 100 - a) for a in np.arange(0, 101, 1)]

In [ ]:
print(s)

In [ ]:
z = np.logspace(-2, 0, num=100)

In [ ]:
print(z)

## reading from .edges

In [ ]:
import pandas as pd
import networkx as nx
import network_diffusion as nd

In [ ]:
def get_ckm_network(path):
    df = pd.read_csv(path, names=["node_1", "node_2", "layer"])
    net_dict = {l_name: nx.Graph() for l_name in [*df["layer"].unique()]}
    for _, row in df.iterrows():
        net_dict[row["layer"]].add_edge(row["node_1"], row["node_2"])
    return nd.MultilayerNetwork.load_layers_nx(
        layer_names=[*net_dict.keys()], network_list=[*net_dict.values()]
    )

In [ ]:
ckm = "data/CKM-Physicians-Innovation_4NoNature.edges"
eu_transportation = "data/EUAirTransportation_multiplex_4NoNature.edges"
lazega = "data/Lazega-Law-Firm_4NoNatureNoLoops.edges"



In [ ]:
net = get_ckm_network(lazega)
print(net.get_actors_num())
print(_ := net.get_nodes_num(), sum(_.values()), len(set(_.keys())))

In [ ]:
def get_aucs_network(file_path):
    return nd.MultilayerNetwork.load_mlx(file_path)

net = get_aucs_network(file_path="data/erererererNoLoops.mpx")
print(net.get_actors_num())
print(_ := net.get_nodes_num(), sum(_.values()), len(set(_.keys())), sum([l.number_of_edges() for l in net.layers.values()]))

## when propagation stopped

In [25]:
import json
import loader
with open("experiment_report/local_stats.json", "r") as f:
    stats = json.load(f)


In [15]:
net = loader.get_aucs_network()

In [26]:
# get last epoch when diffusion took place and final coverage in the network

length_of_diffusion = 0
activated_nodes_list = []

for epoch_num, epoch_changes in stats.items():
    if len(epoch_changes) > 0:
        length_of_diffusion = int(epoch_num) + 1 # because of seeding epoch
    for node in epoch_changes:
        if node["new_state"] == "1":
            activated_nodes_list.append(node['node_name'])
        else:  # sanity check to detect leaks when nodes gets deactivated
            if int(epoch_num) > 0:
                raise AttributeError("Results are contradicting themselves!")

activated_actors = len(set(activated_nodes_list))

length_of_diffusion, activated_actors

(4, 61)

In [6]:
# get the coverage in the network
net.get_actors_num()

61

In [22]:
activated_actors =[]
for k, v in stats.items():
    print(k)
    for node in v:
        if node["new_state"] == "1":
            activated_actors.append(node['node_name'])
        else:
            if int(k) > 0:
                raise AttributeError("Something went wrokg - results are contradicting themselves")

print(len(activated_actors))
activated_actors = set(activated_actors)
print(len(activated_actors))

0
1
2
4
5
6
7
8
221
60
